In [1]:
from motion.BVH import BVH as BVH
from net.quat import quat
from net.euler import euler
from preprocess.fbx2bvh import file_conversion
import numpy as np
import re
import copy
import math
import os

human_data = []
animal_data = []

#file_conversion()

In [2]:
def load_data(dir, array):
    for root, _, files in os.walk(dir):
        for file in files:
            skeleton = BVH.load(dir + file)
            #skeleton.calculate_global_positions(skeleton.root)
            array.append(skeleton)


In [3]:
load_data("./data/Datasets/Walk/Human/", human_data)
load_data("./data/Datasets/Walk/Animal/", animal_data)

In [4]:
def scale_centaur(node, num):
    for child in node:
        child.offsets = [x * num for x in child.offsets]
        scale_centaur(child.children, num) 

def create_centaur(human_skeleton, horse_skeleton):
    centaur_skeleton = copy.deepcopy(human_skeleton)
    centaur_skeleton.root = copy.deepcopy(horse_skeleton.root)
    centaur_skeleton.root.children[0].children[0].children[0].children[0].children[0].children[0] = copy.deepcopy(human_skeleton.root.children[2])
    scale_centaur(centaur_skeleton.root.children[0].children[0].children[0].children[0].children[0].children[0].children, 9)

    human_skeleton.calculate_global_positions(human_skeleton.root)
    horse_skeleton.calculate_global_positions(horse_skeleton.root)
    centaur_skeleton.calculate_global_positions(centaur_skeleton.root)

In [5]:
def compareFiles():
    import difflib
    
    # Function to read a file, remove tabs, and return its content as a list of lines
    def read_file(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            # Read the content and replace tabs with spaces
            content = file.read().replace('\t', '')
        # Split the content into lines
        lines = content.split('\n')
        return lines
    
    # Paths to the two text files you want to compare
    file1_path = './data/Datasets/Walk/Animal/AlligatorALL-Walk1.bvh'
    file2_path = 'animal.bvh'
    
    # Read and preprocess the content of both files
    file1_lines = read_file(file1_path)
    file2_lines = read_file(file2_path)
    
    # Compute the differences using difflib
    differ = difflib.Differ()
    diff = list(differ.compare(file1_lines, file2_lines))
    
    # Display the differences
    for line in diff:
        print(line)

In [6]:
index = 80
BVH.save(animal_data[index].filename, animal_data[index])

In [44]:
def get_feature_vectors(skeleton):
    all_feature_vec = []
    for name in skeleton.nodes:
        feature_vec = []
        for frame in range(len(skeleton.nodes[name].positions_local)):
            if(len(skeleton.nodes[name].positions_local) > 0):
                feature_vec.append(np.concatenate((skeleton.nodes[name].positions_local[frame], skeleton.nodes[name].velocities[frame]), axis=None))
        if feature_vec: all_feature_vec.append(feature_vec)
    return all_feature_vec

human_feature_vectors = []
animal_feature_vectors = []

for value in animal_data:
    animal_feature_vectors.append(get_feature_vectors(value))
for value in human_data:
    human_feature_vectors.append(get_feature_vectors(value))

In [45]:
Maxlen = 0
for animal in animal_feature_vectors:
    if (len(animal)) > Maxlen : Maxlen = (len(animal))

In [119]:
padded_animal_feature_vect = []
padded_human_feature_vect = []

for animal in animal_feature_vectors:
    #print(np.pad(animal, ((0, Maxlen - len(animal)),(0,0),(0,0)),mode='constant').shape)
    padded_animal_feature_vect.append(np.pad(animal, ((0, Maxlen - len(animal)),(0,0),(0,0)),mode='constant'))

for human in human_feature_vectors:
    #print(np.pad(animal, ((0, Maxlen - len(animal)),(0,0),(0,0)),mode='constant').shape)
    padded_human_feature_vect.append(np.pad(human, ((0, Maxlen - len(human)),(0,0),(0,0)),mode='constant'))



In [123]:
#
#
# TODO: LOOP ANIMATIONS TO HAVE MATCHING FRAMERATES
#
#

# Example feature vectors
feature_vector1 = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])  # Shape: (2, 3)
feature_vector2 = np.array([[7.0, 8.0, 9.0], [10.0, 11.0, 12.0]])  # Shape: (2, 3)

all_feature_vector = []

for human in padded_human_feature_vect:
    for animal in padded_animal_feature_vect:
        # Concatenate the feature vectors along a new dimension (axis=1)
        all_feature_vector.append(np.concatenate((human[:, np.newaxis, :], animal[:, np.newaxis, :]), axis=1))

print(len(all_feature_vector))

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 2, the array at index 0 has size 60 and the array at index 1 has size 231

In [68]:
import torch
import torch.nn as nn
import torch.optim as optim
import time, random, math, string

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, num_layers=n_layers, dropout=dropout)
        
        self.dropout = nn.Dropout(dropout)
    def forward(self, src):
        # src : [sen_len, batch_size]
        embedded = self.dropout(self.embedding(src))
        
        # embedded : [sen_len, batch_size, emb_dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        # outputs = [sen_len, batch_size, hid_dim * n_directions]
        # hidden = [n_layers * n_direction, batch_size, hid_dim]
        # cell = [n_layers * n_direction, batch_size, hid_dim]
        return hidden, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, num_layers=self.n_layers, dropout=dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        # input = [batch_size]
        # hidden = [n_layers * n_dir, batch_size, hid_dim]
        # cell = [n_layers * n_dir, batch_size, hid_dim]
        
        input = input.unsqueeze(0)
        # input : [1, ,batch_size]
        
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch_size, emb_dim]
        
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [seq_len, batch_size, hid_dim * n_dir]
        # hidden = [n_layers * n_dir, batch_size, hid_dim]
        # cell = [n_layers * n_dir, batch_size, hid_dim]
        
        # seq_len and n_dir will always be 1 in the decoder
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch_size, output_dim]
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            'hidden dimensions of encoder and decoder must be equal.'
        assert encoder.n_layers == decoder.n_layers, \
            'n_layers of encoder and decoder must be equal.'
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        # src = [sen_len, batch_size]
        # trg = [sen_len, batch_size]
        # teacher_forcing_ratio : the probability to use the teacher forcing.
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        # tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        # last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        # first input to the decoder is the <sos> token.
        input = trg[0, :]
        for t in range(1, trg_len):
            # insert input token embedding, previous hidden and previous cell states 
            # receive output tensor (predictions) and new hidden and cell states.
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            # replace predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            # decide if we are going to use teacher forcing or not.
            teacher_force = random.random() < teacher_forcing_ratio
            
            # get the highest predicted token from our predictions.
            top1 = output.argmax(1)
            # update input : use ground_truth when teacher_force 
            input = trg[t] if teacher_force else top1
            
        return outputs

In [ ]:
# First initialize our model.
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(encoder, decoder, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
optimizer = optim.Adam(model.parameters())

TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        # trg = [sen_len, batch_size]
        # output = [trg_len, batch_size, output_dim]
        output = model(src, trg)
        output_dim = output.shape[-1]
        
        # transfrom our output : slice off the first column, and flatten the output into 2 dim.
        output = output[1:].view(-1, output_dim) 
        trg = trg[1:].view(-1)
        # trg = [(trg_len-1) * batch_size]
        # output = [(trg_len-1) * batch_size, output_dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
        
        for i, batch in enumerate(iterator):
            
            src = batch.src
            trg = batch.trg
            
            output = model(src, trg, 0) # turn off teacher forcing.
            
            # trg = [sen_len, batch_size]
            # output = [sen_len, batch_size, output_dim]
            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            
            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
            
    return epoch_loss / len(iterator)

In [ ]:
# a function that used to tell us how long an epoch takes.
def epoch_time(start_time, end_time):
    
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time  / 60)
    elapsed_secs = int(elapsed_time -  (elapsed_mins * 60))
    return  elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10

CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'Seq2SeqModel.pt')
    print(f"Epoch: {epoch+1:02} | Time {epoch_mins}m {epoch_secs}s")
    print(f"\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}")
    print(f"\tValid Loss: {valid_loss:.3f} | Valid PPL: {math.exp(valid_loss):7.3f}")

In [ ]:
def test():
    best_model = Seq2Seq(encoder, decoder, device).to(device)
    best_model.load_state_dict(torch.load('Seq2SeqModel.pt'))
    
    test_loss = evaluate(model, test_iter, criterion)
    
    print(f"Test Loss : {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f}")
    
test()